In [1]:
# - *- coding: utf- 8*-
from LambdaNodeEmbedding import KerasEval
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.callbacks import History

#These are the edges between the nodes.
test_0 = np.array([
                    [0,0.3,1,1],
                    [0,0.1,0,1],
                    [0,1,0,0.2],
                    [0,0,0,0.5]
                    ], dtype='float32')

test_1 = np.array([
                    [0,1,1,1],
                    [0,0,0,1],
                    [0,1,0,0],
                    [0,0,0,0]
                    ], dtype='float32')

weights_0 = np.array([
                    [0,1,1,1,0,0,0.3,0],
                    [0,0,0,1,1,0,0.3,0],
                    [0,1,0,0,0,1,0.3,0],
                    [0,0,0,0,1,0,0.3,0]
                    ], dtype='float32')

weights_1 = np.array([
                    [0,1,1,0.1,0,0,0.1,],
                    [0,0,0,0.2,1,0,0.1,],
                    [0,0.3,0,0,0,1,0.1,],
                    [0,0,0,0,0.5,0,0.1,]
                    ], dtype='float32')

test_0_ten = tf.keras.backend.variable (value=test_0, dtype='float32')
test_1_ten = tf.keras.backend.variable (value=test_1, dtype='float32')
weigts_0_ten = tf.keras.backend.variable (value=weights_0, dtype='float32')
weigts_1_ten = tf.keras.backend.variable (value=weights_1, dtype='float32')

#edge_len = tf.keras.backend.constant (value=edgelist_ten.shape[1], dtype='int32')
#at_least_one = tf.keras.backend.variable (value=1, dtype='int32')

def GetXDepth(matrix):
    if len(matrix.shape) > 0:
        return matrix.shape[0]
    else:
        return 1

def GetYDepth(matrix):
    if len(matrix.shape) > 1:
        return matrix.shape[1]
    else:
        return 1

def BuildZeroedConcatRow(edge_row, index):
    zero_calc = tf.zeros((index,), tf.float32)
    keep_calc =  tf.ones(((edge_row.shape[0]-index),), tf.float32)
    concat_calc = tf.concat([zero_calc,keep_calc],0)
    result = tf.multiply(concat_calc, edge_row) 
    return result

def IndexOutOfBounds(edge_row, index):
    return tf.cond(tf.less_equal(index, edge_row.shape[0]), 
                   lambda: BuildZeroedConcatRow(edge_row, index),
                   lambda: tf.zeros(edge_row.shape, tf.float32))

def ZerofyUnderIndexInRow(edge_row, index):
    return tf.cond(tf.less(index, at_least_one), 
                   lambda: edge_row,
                   lambda: IndexOutOfBounds(edge_row, index))

def ZerofyUnderPrincipalDiagonal(edge_emb, result_emb, iterator):
    
    #Condition => if current iteration is lower as max iterations
    def cond(edge_emb, result_emb, i):
         return tf.less(i, iterator)

    #Loop Body Processing Unit => get all connection dependencies
    def body(edge_emb, result_emb, i):
        result_row = ZerofyUnderIndexInRow(edge_emb[i,:], i)
        
        #Store result in tensor array result_emb
        result_emb = result_row.write(i, [result_row])
        
        #Add +1 to Iterator and return process result!
        return [edge_emb, result_emb, tf.add(i, 1)]

    #return the loop setup
    return tf.while_loop(cond, body, [edge_emb, result_emb, 0])


def Call_Function(self_node_vecs, neigh_node_vecs, self_node_weights, neigh_node_weights, neigh_input_dim=4, hidden_dim=8):
    batch_size = neigh_node_vecs.shape[0]
    num_neighbors = neigh_node_vecs.shape[1]
    new_dim = batch_size * num_neighbors
    
    print('Batch_Size: ', batch_size)
    print('Neighbours: ', num_neighbors)
    print('Conversion: Shape_Neighbours -> ', neigh_node_vecs.shape, 'to (',new_dim, ', ', neigh_input_dim,')')
    
    """ [1] """
    hidden_reshaped = K.reshape(neigh_node_vecs, (new_dim, neigh_input_dim))
    hidden_reshaped = l(hidden_reshaped)
        
    """ [2] """
    neigh_node_vecs = K.reshape(hidden_reshaped, (batch_size, num_neighbors, hidden_dim))
    neigh_node_vecs = K.max(neigh_node_vecs, axis=1)
    
    """ [3] """
    from_neighs = K.dot(neigh_node_vecs, neigh_node_weights)
    from_self = K.dot(self_node_vecs, self_node_weights)

    """ [4] """
    output = (from_self + from_neighs) if (not self.concat) else K.concatenate([from_self, from_neighs], axis=1)



init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    trst = Call_Function(test_0_ten, test_1_ten, weigts_0_ten, weigts_1_ten)
    
    
    #my_tuple = [test_0_ten,test_0_ten]
    #tf_add_test = tf.add_n(my_tuple)
    #print('Result_T:', KerasEval(tf_add_test))
    #print('')
    #keras_sum_test = my_tuple[0] + my_tuple[1]
    #print('Result_K: ', KerasEval(keras_sum_test))
    
    #mult_concat_t0_dot_1st = K.dot(test_0_ten, weigts_0_ten)
    #mult_concat_t1_dot_1st = K.dot(test_1_ten, weigts_0_ten)
    #mult_concat_final_dot_1st = K.concatenate([mult_concat_t0_dot_1st, mult_concat_t1_dot_1st], axis=1)
    #print('Result_MC_Dot:\n', KerasEval(mult_concat_t0_dot_1st))
    #print('Result_MC_Dot:\n', KerasEval(mult_concat_t1_dot_1st))
    
    #print('Final_MC:\n', KerasEval(mult_concat_final_dot_1st))
    
    
    #concat_mult_cc = K.concatenate([test_0_ten, test_1_ten], axis=1)    
    #concat_mult_dot = K.dot(concat_mult_cc, weigts_0_ten)
    
    #print('Result_CM_CC:\n', KerasEval(concat_mult_cc))
    #print('Final_CM:\n', KerasEval(concat_mult_dot))
    
    #equalitiy_check = (concat_mult_dot == mult_concat_final_dot_1st)
    #print('Check: ', equalitiy_check)
    
    print('Test: ', KerasEval(test))
    
    
    


Using TensorFlow backend.


Batch_Size:  4
Neighbours:  4
New_Dim:  16
Conversion: Shape_Neighbours ->  (4, 4) to ( 16 4 )


ValueError: Cannot reshape a tensor with 16 elements to shape [16,4] (64 elements) for 'Reshape' (op: 'Reshape') with input shapes: [4,4], [2] and with input tensors computed as partial shapes: input[1] = [16,4].